In [1]:
from os import name
import sys
import tensorflow as tf
from tensorflow.keras import backend as K
tf.keras.backend.set_floatx('float32')

from tensorflow.keras import Input
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, UpSampling2D, Concatenate, BatchNormalization, Conv2DTranspose, Flatten, PReLU, Reshape, Dropout, AveragePooling2D, Add, Lambda, Layer
from tensorflow.keras.regularizers import l2
from tensorflow.keras.initializers import Constant
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model

import h5py
import numpy as np
import MD_AE_model as model

2022-03-22 15:03:10.769887: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-03-22 15:03:10.769931: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
# data
data_file = './PIV4_downsampled_by8.h5'
Ntrain = 10 # snapshots for training
Nval = 600 # sanpshots for validation
Ntest = 600

# Boolean 
LATENT_STATE = True # save latent state
SHUFFLE = True # shuffle before splitting into sets, test set is extracted before shuffling
REMOVE_MEAN = True # train on fluctuating velocity

## ae configuration
lmb = 0.0 #1e-05 #regulariser
drop_rate = 0.2
features_layers = [32, 64, 128]
# latent_dim = 10
batch_size = 5
act_fct = 'tanh'
resize_meth = 'bilinear'
filter_window= (3,3)
batch_norm = False

Nz = 24 # grid size
Ny = 21
Nu = 2
Nt = 2732 # number of snapshots available
D = 196.5 # mm diameter of bluff body
U_inf = 15 # m/s freestream velocity
f_piv = 720.0 # Hz PIV sampling frequency  
dt = 1.0/f_piv 

print('Reading dataset from :' + data_file)
hf = h5py.File(data_file,'r')

z = np.array(hf.get('z'))
y = np.array(hf.get('y'))
u_all = np.zeros((Nt,Nz,Ny,Nu))
u_all[:,:,:,0] = np.array(hf.get('vy'))
if Nu==2:
    u_all[:,:,:,1] = np.array(hf.get('vz'))
u_all = np.transpose(u_all,[0,2,1,3]) # shape of u_all = (Nt,Ny,Nz,Nu)
hf.close()

u_all = u_all[:,:,:,:].astype('float32')

# remove mean for modes
if REMOVE_MEAN:
    u_mean_all = np.mean(u_all,axis=0) # time averaged, (Ny,Nz,Nu)
    u_all = u_all - u_mean_all


if SHUFFLE:
    # temp_list = list(u_all)
    # np.random.shuffle(temp_list) # this shuffles the first axis
    # u_all = np.array(temp_list)

    idx_test = np.random.randint(0,Nt-Ntest)
    u_test = u_all[idx_test:idx_test+Ntest,:,:,:].astype('float32') # test set needs to be in order and has continuous snapshots
    u_all = np.delete(u_all,np.s_[idx_test:idx_test+Ntest],0) # remove the test set from available samples
    idx_shuffle = np.arange(Nt-Ntest) # use idx_shuffle to shuffle the rest of samples before taking a validation set
    np.random.shuffle(idx_shuffle)
    idx_unshuffle = np.argsort(idx_shuffle) # use idx_unshuffle to unshuffle the data
    u_all = u_all[idx_shuffle,:,:,:]
    u_train = u_all[0:Ntrain,:,:,:].astype('float32')
    u_val = u_all[Ntrain:Ntrain+Nval,:,:,:].astype('float32')
    u_all = np.vstack((u_train,u_val,u_test))
else:
    u_train = u_all[0:Ntrain,:,:,:].astype('float32')
    u_val = u_all[Ntrain:Ntrain+Nval,:,:,:].astype('float32')
    u_test = u_all[Ntrain+Nval:Ntrain+Nval+Ntest,:,:,:].astype('float32')
    u_all = u_all[0:Ntrain+Nval+Ntest,:,:,:].astype('float32') # u_all has shape (Ntrain+Nval+Ntest,Ny,Nz,Nu)


u_all = np.reshape(u_all,(1,Ntrain+Nval+Ntest,Ny,Nz,Nu)) # new shape (1,Nval+Ntrain+Ntest,Ny,Nz,Nu)
u_train = np.reshape(u_train,(1,Ntrain,Ny,Nz,Nu))
u_val = np.reshape(u_val,(1,Nval,Ny,Nz,Nu))
u_test = np.reshape(u_test,(1,Ntest,Ny,Nz,Nu))
Nx = [Ny, Nz]

Reading dataset from :./PIV4_downsampled_by8.h5


In [3]:
u = u_train[0,:,:,:,:]
print(u.shape)

(10, 21, 24, 2)


In [4]:
encoder1 = model.Encoder(Nx=Nx,Nu=Nu,features_layers=features_layers,latent_dim=1,filter_window=filter_window,act_fct=act_fct,batch_norm=batch_norm,drop_rate=drop_rate,lmb=lmb)
layer_size = encoder1.get_layer_shape()

input_img = [Input(shape=(Nx[0],Nx[1],Nu))]
x = encoder1(input_img[0])
print(len(input_img) != 1)
x_final = model.Decoder(Nx,Nu,layer_size,features_layers=features_layers,latent_dim=1,filter_window=filter_window,act_fct=act_fct,batch_norm=batch_norm,drop_rate=drop_rate,lmb=lmb)(x)
mdl1 = Model(input_img,x_final)
print(mdl1.summary())
mdl1.compile(optimizer=Adam(learning_rate=0.001),loss='mse')
mdl1.fit([u],u,epochs=2,batch_size=batch_size)
z1 = encoder1.predict(u)
print(z1.shape)

2022-03-22 15:03:13.063178: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-03-22 15:03:13.063203: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-03-22 15:03:13.063221: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (ic-5rpzl43): /proc/driver/nvidia/version does not exist
2022-03-22 15:03:13.063448: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


False
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 21, 24, 2)]       0         
                                                                 
 encoder (Encoder)           (None, 1)                 94113     
                                                                 
 decoder (Decoder)           (None, 21, 24, 2)         95138     
                                                                 
Total params: 189,251
Trainable params: 189,251
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/2
2/2 [==============================] - 1s 8ms/step - loss: 2.0114
Epoch 2/2
2/2 [==============================] - 0s 9ms/step - loss: 1.9877
(10, 1)


In [5]:
input_img = input_img = [Input(shape=(Nx[0],Nx[1],Nu))]
input_img.extend([Input(shape=1)])
print(input_img) 
print(len(input_img) != 1)

[<KerasTensor: shape=(None, 21, 24, 2) dtype=float32 (created by layer 'input_4')>, <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'input_5')>]
True


In [6]:
encoder2 = model.Encoder(Nx=Nx,Nu=Nu,features_layers=features_layers,latent_dim=1,filter_window=filter_window,act_fct=act_fct,batch_norm=batch_norm,drop_rate=drop_rate,lmb=lmb)
layer_size2 = encoder2.get_layer_shape()

new_dim = np.sum([1],dtype=np.dtype('int'))+1
print(new_dim)

x = encoder2(input_img[0])
new_x = input_img[1::]
new_x.extend([x])
print(type(new_x))
x = Concatenate(name='stack')(new_x)
print(x)

x_final = model.Decoder(Nx,Nu,layer_size,features_layers=features_layers,latent_dim=new_dim,filter_window=filter_window,act_fct=act_fct,batch_norm=batch_norm,drop_rate=drop_rate,lmb=lmb)(x)
mdl2 = Model(input_img,x_final)
print(mdl2.summary())
mdl2.compile(optimizer=Adam(learning_rate=0.001),loss='mse')
mdl2.fit([u,z1],u,epochs=2,batch_size=batch_size)
z2 = encoder2.predict(u)

2
<class 'list'>
KerasTensor(type_spec=TensorSpec(shape=(None, 2), dtype=tf.float32, name=None), name='stack/concat:0', description="created by layer 'stack'")
Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, 21, 24, 2)]  0           []                               
                                                                                                  
 input_5 (InputLayer)           [(None, 1)]          0           []                               
                                                                                                  
 encoder_1 (Encoder)            (None, 1)            94113       ['input_4[0][0]']                
                                                                                                  
 stack (Concatenate)           

In [ ]:
input_img = input_img = [Input(shape=(Nx[0],Nx[1],Nu))]
input_img.extend([Input(shape=1)])
print(input_img) 

In [ ]:
encoder3 = model.Encoder(Nx=Nx,Nu=Nu,features_layers=features_layers,latent_dim=1,filter_window=filter_window,act_fct=act_fct,batch_norm=batch_norm,drop_rate=drop_rate,lmb=lmb)
layer_size3 = encoder3.get_layer_shape()

new_dim = np.sum([1,1],dtype=np.dtype('int'))+1
print(new_dim)

x = encoder3(input_img[0])
new_x = input_img[1::]
new_x.extend([x])
print(type(new_x))
x = Concatenate(name='stack')(new_x)
print(x)

x_final = model.Decoder(Nx,Nu,layer_size,features_layers=features_layers,latent_dim=new_dim,filter_window=filter_window,act_fct=act_fct,batch_norm=batch_norm,drop_rate=drop_rate,lmb=lmb)(x)
mdl2 = Model(input_img,x_final)
print(mdl2.summary())
mdl2.compile(optimizer=Adam(learning_rate=0.001),loss='mse')
mdl2.fit([u,z1],u,epochs=2,batch_size=batch_size)
z2 = encoder2.predict(u)